# Confluence OAuth 인증 테스트

이 노트북은 Atlassian Confluence API에 OAuth 인증을 통해 접근하는 과정을 단계별로 보여줍니다.

## 필요한 패키지 설치


In [ ]:
# 필요한 패키지 설치
# %pip install requests python-dotenv


## 1단계: 환경 설정

먼저 `.env` 파일을 생성하고 OAuth 설정을 확인합니다.


In [ ]:
import os
import requests
import webbrowser
from urllib.parse import urlencode, parse_qs, urlparse
from dotenv import load_dotenv
import json
import time

# .env 파일 로드
load_dotenv('.env')

# OAuth 설정
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = "http://localhost:8000/callback"
SCOPE = (
    "read:page:confluence "
    "write:page:confluence "
    "delete:page:confluence "
    "read:content:confluence "
    "write:content:confluence "
    "read:content-details:confluence "
    "read:space:confluence "
    "read:me "
    "offline_access"
)

# Confluence 설정
ATLASSIAN_SITE = os.getenv("ATLASSIAN_SITE", "https://your-site.atlassian.net")
CLOUD_ID = os.getenv("CLOUD_ID")
SPACE_ID = int(os.getenv("SPACE_ID", "0"))
PARENT_ID = int(os.getenv("PARENT_ID", "0"))
SPACE_KEY = os.getenv("SPACE_KEY")
CONFLUENCE_RESOURCE_NAME = os.getenv("CONFLUENCE_RESOURCE_NAME", "ktspace")

print("=== 환경 설정 확인 ===")
print(f"Client ID: {CLIENT_ID}")
print(f"Client Secret: {'설정됨' if CLIENT_SECRET else '❌ 설정되지 않음'}")
print(f"Redirect URI: {REDIRECT_URI}")
print(f"Scope: {SCOPE}")
print(f"Atlassian Site: {ATLASSIAN_SITE}")
print(f"Cloud ID: {CLOUD_ID}")
print(f"Space ID: {SPACE_ID}")
print(f"Parent ID: {PARENT_ID}")
print(f"Space Key: SPACE_KEY")
print(f"Confluence Resource Name: CONFLUENCE_RESOURCE_NAME")

# 필수 환경변수 확인
missing_vars = []
if not CLIENT_SECRET:
    missing_vars.append("CLIENT_SECRET")
if not CLOUD_ID:
    missing_vars.append("CLOUD_ID")
if not SPACE_ID:
    missing_vars.append("SPACE_ID")
if not PARENT_ID:
    missing_vars.append("PARENT_ID")

if missing_vars:
    print(f"\n❌ 다음 환경변수들이 .env 파일에 설정되지 않았습니다: {', '.join(missing_vars)}")
    print("   .env 파일에 해당 값들을 추가해주세요.")


## 2단계: 콜백 서버 실행

인증 후 돌아올 콜백 서버를 먼저 실행합니다.


In [ ]:
import http.server
import socketserver
import urllib.parse
import threading
import time

class CallbackHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path.startswith('/callback'):
            # URL 파라미터 파싱
            parsed_url = urllib.parse.urlparse(self.path)
            params = urllib.parse.parse_qs(parsed_url.query)
            
            if 'code' in params:
                code = params['code'][0]
                print(f"\n✅ 인증 코드를 받았습니다: {code}")
                
                # 전역 변수에 저장
                global authorization_code
                authorization_code = code
                
                # 성공 페이지 응답
                self.send_response(200)
                self.send_header('Content-type', 'text/html; charset=utf-8')
                self.end_headers()
                
                html_content = f"""
                <html>
                <body>
                    <h2>✅ 인증 성공!</h2>
                    <p>인증 코드: <strong>{code}</strong></p>
                    <p>이제 Jupyter 노트북으로 돌아가서 다음 단계를 진행하세요.</p>
                </body>
                </html>
                """
                
                self.wfile.write(html_content.encode('utf-8'))
                
                # 서버 종료
                threading.Thread(target=self.server.shutdown).start()
            else:
                # 오류 페이지
                self.send_response(400)
                self.send_header('Content-type', 'text/html; charset=utf-8')
                self.end_headers()
                
                html_content = """
                <html>
                <body>
                    <h2>❌ 인증 실패</h2>
                    <p>인증 코드를 받지 못했습니다.</p>
                </body>
                </html>
                """
                
                self.wfile.write(html_content.encode('utf-8'))
        else:
            self.send_response(404)
            self.end_headers()

def start_callback_server():
    PORT = 8000
    with socketserver.TCPServer(("", PORT), CallbackHandler) as httpd:
        print(f"🔄 콜백 서버가 http://localhost:{PORT} 에서 실행 중...")
        print("브라우저에서 OAuth URL을 열어주세요.")
        httpd.serve_forever()

# 콜백 서버를 백그라운드에서 실행
server_thread = threading.Thread(target=start_callback_server, daemon=True)
server_thread.start()

print("✅ 콜백 서버가 시작되었습니다.")
print("이제 다음 단계에서 OAuth URL을 생성하고 브라우저에서 열어주세요!")


## 3단계: OAuth 인증 URL 생성

콜백 서버가 실행된 상태에서 OAuth 인증 URL을 생성하고 브라우저에서 엽니다.


In [ ]:
def get_authorization_url():
    """OAuth 인증 URL 생성 (read:me 스코프 포함)"""
    # read:me 스코프를 포함한 전체 스코프
    full_scope = SCOPE
    params = {
        "audience": "api.atlassian.com",
        "client_id": CLIENT_ID,
        "scope": full_scope,
        "redirect_uri": REDIRECT_URI,
        "state": "random_state_string",
        "response_type": "code",
        "prompt": "consent"
    }
    
    auth_url = f"https://auth.atlassian.com/authorize?{urlencode(params)}"
    return auth_url

# 인증 URL 생성
auth_url = get_authorization_url()
print("=== OAuth 인증 URL (read:me 스코프 포함) ===")
print(auth_url)
print("\n🔗 위 URL을 브라우저에서 열어주세요!")
print("   (자동으로 브라우저가 열리지 않으면 수동으로 복사해서 열어주세요)")

# 브라우저 자동 열기 시도
try:
    webbrowser.open(auth_url)
    print("✅ 브라우저가 자동으로 열렸습니다.")
except:
    print("⚠️  브라우저를 수동으로 열어주세요.")

print("\n📝 인증 완료 후 콜백 서버에서 인증 코드를 받으면 다음 단계로 진행하세요.")


## 4단계: 액세스 토큰 발급 및 관리

인증 코드를 액세스 토큰으로 교환하고 로컬 변수로 관리합니다.


In [ ]:
import ssl
import urllib3
import base64
import json
import datetime

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 저장된 인증 코드 확인 및 액세스 토큰 발급
try:
    if 'authorization_code' in globals() and authorization_code:
        print(f"✅ 저장된 인증 코드: {authorization_code}")
        
        # 액세스 토큰으로 교환 (SSL 검증 우회)
        def exchange_code_for_token(authorization_code):
            token_url = "https://auth.atlassian.com/oauth/token"
            
            data = {
                "grant_type": "authorization_code",
                "client_id": CLIENT_ID,
                "client_secret": CLIENT_SECRET,
                "code": authorization_code,
                "redirect_uri": REDIRECT_URI
            }
            
            print("🔄 액세스 토큰을 발급받는 중...")
            
            # SSL 검증을 우회하여 요청
            response = requests.post(token_url, data=data, verify=False)
            
            print(f"응답 상태 코드: {response.status_code}")
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"응답 내용: {response.text}")
                response.raise_for_status()
        
        # 토큰 교환 실행
        token_response = exchange_code_for_token(authorization_code)
        access_token = token_response.get("access_token")
        
        if access_token:
            print(f"\n✅ 액세스 토큰 발급 성공!")
            print(f"액세스 토큰: {access_token[:20]}...")
            
            # 전역 변수에 저장 (로컬 변수로 관리)
            oauth_access_token = access_token
            
            # 토큰 만료 시간 확인
            def decode_jwt_token(token):
                """JWT 토큰 디코딩 (페이로드 부분만)"""
                try:
                    parts = token.split('.')
                    if len(parts) != 3:
                        return None
                    payload = parts[1]
                    payload += '=' * (4 - len(payload) % 4)
                    decoded = base64.urlsafe_b64decode(payload)
                    return json.loads(decoded)
                except:
                    return None
            
            token_info = decode_jwt_token(access_token)
            if token_info:
                exp_timestamp = token_info.get('exp')
                if exp_timestamp:
                    exp_time = datetime.datetime.fromtimestamp(exp_timestamp)
                    current_time = datetime.datetime.now()
                    print(f"\n📅 토큰 만료 시간: {exp_time}")
                    print(f"⏰ 현재 시간: {current_time}")
                    print(f"⏳ 만료까지 남은 시간: {exp_time - current_time}")
            
            print(f"\n💡 이 토큰은 로컬 변수로 관리됩니다 (1시간 후 만료)")
            
        else:
            print("❌ 액세스 토큰을 받지 못했습니다.")
            
    else:
        print("❌ 인증 코드가 저장되지 않았습니다.")
        print("콜백 서버를 다시 실행하고 브라우저에서 OAuth URL을 열어주세요.")
        
except NameError:
    print("❌ authorization_code 변수가 정의되지 않았습니다.")
    print("콜백 서버를 다시 실행하고 브라우저에서 OAuth URL을 열어주세요.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")


## 5단계: Confluence API 테스트

발급받은 액세스 토큰을 사용해서 Confluence API에 접근해봅니다.


In [ ]:
def test_confluence_api(access_token):
    """Confluence API 테스트"""
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # 1. 사용자 정보 조회
    print("=== 1. 사용자 정보 조회 ===")
    try:
        user_url = "https://api.atlassian.com/me"
        response = requests.get(user_url, headers=headers, verify=False)
        
        if response.status_code == 200:
            user_info = response.json()
            print(f"✅ 사용자 정보 조회 성공")
            print(f"사용자 ID: {user_info.get('account_id')}")
            print(f"이메일: {user_info.get('email')}")
            print(f"이름: {user_info.get('name')}")
        else:
            print(f"❌ 사용자 정보 조회 실패: {response.status_code}")
            print(f"응답: {response.text}")
    except Exception as e:
        print(f"❌ 사용자 정보 조회 중 오류: {e}")
    
    # 2. Confluence 사이트 정보 조회
    print("\n=== 2. Confluence 사이트 정보 조회 ===")
    try:
        # Cloud ID 조회
        cloud_id_url = f"https://api.atlassian.com/oauth/token/accessible-resources"
        response = requests.get(cloud_id_url, headers=headers, verify=False)
        
        if response.status_code == 200:
            resources = response.json()
            print(f"✅ 접근 가능한 리소스 조회 성공")
            print(f"총 리소스 수: {len(resources)}")
            
            # 모든 리소스 정보 출력
            print("\n📋 접근 가능한 모든 리소스:")
            for i, resource in enumerate(resources):
                print(f"  {i+1}. 이름: {resource.get('name')}")
                print(f"     ID: {resource.get('id')}")
                print(f"     URL: {resource.get('url')}")
                print(f"     타입: {resource.get('type')}")
                print()
            
            # Confluence 리소스 찾기 (환경변수 기반)
            confluence_resource = None
            confluence_names = ['Confluence', 'confluence', 'CONFLUENCE', 'Wiki', 'wiki', CONFLUENCE_RESOURCE_NAME, 'KTSPACE']
            
            for resource in resources:
                if resource.get('name') in confluence_names:
                    confluence_resource = resource
                    cloud_id = resource.get('id')
                    print(f"✅ Confluence 리소스 발견!")
                    print(f"리소스 이름: {resource.get('name')}")
                    print(f"Cloud ID: {cloud_id}")
                    print(f"사이트 URL: {resource.get('url')}")
                    break
            
            if confluence_resource:
                # 3. Confluence 스페이스 조회
                print("\n=== 3. Confluence 스페이스 조회 ===")
                spaces_url = f"https://api.atlassian.com/ex/confluence/{cloud_id}/wiki/api/v2/spaces"
                spaces_response = requests.get(spaces_url, headers=headers, verify=False)
                
                if spaces_response.status_code == 200:
                    spaces_data = spaces_response.json()
                    print(f"✅ 스페이스 조회 성공")
                    print(f"총 스페이스 수: {spaces_data.get('size', 0)}")
                    
                    # 첫 번째 스페이스 정보 출력
                    if spaces_data.get('results'):
                        first_space = spaces_data['results'][0]
                        print(f"첫 번째 스페이스: {first_space.get('name')} (ID: {first_space.get('id')})")
                        print(f"스페이스 키: {first_space.get('key')}")
                else:
                    print(f"❌ 스페이스 조회 실패: {spaces_response.status_code}")
                    print(f"응답: {spaces_response.text}")
            else:
                print("❌ Confluence 리소스를 찾을 수 없습니다.")
                print("💡 OAuth 앱이 Confluence 사이트에 연결되지 않았을 수 있습니다.")
                print("   Atlassian Console에서 OAuth 앱 설정을 확인해주세요.")
        else:
            print(f"❌ 리소스 조회 실패: {response.status_code}")
            print(f"응답: {response.text}")
            
    except Exception as e:
        print(f"❌ Confluence API 테스트 중 오류: {e}")

# API 테스트 실행
if 'oauth_access_token' in globals() and oauth_access_token:
    test_confluence_api(oauth_access_token)
    
    # 추가: SPACE_KEY 스페이스 확인
    print("\n=== 4. SPACE_KEY 스페이스 확인 ===")
    headers = {
        "Authorization": f"Bearer {oauth_access_token}",
        "Content-Type": "application/json"
    }
    
    # Cloud ID 가져오기
    try:
        cloud_id_url = f"https://api.atlassian.com/oauth/token/accessible-resources"
        response = requests.get(cloud_id_url, headers=headers, verify=False)
        
        if response.status_code == 200:
            resources = response.json()
            cloud_id = None
            for resource in resources:
                if resource.get('name') == CONFLUENCE_RESOURCE_NAME:
                    cloud_id = resource.get('id')
                    break
            
            if cloud_id:
                # v2 API로 SPACE_KEY 스페이스 조회
                print("v2 API로 SPACE_KEY 스페이스 조회 시도...")
                v2_url = f"https://api.atlassian.com/ex/confluence/{cloud_id}/wiki/api/v2/spaces?keys=SPACE_KEY"
                v2_response = requests.get(v2_url, headers=headers, verify=False)
                print(f"v2 API 응답: {v2_response.status_code}")
                if v2_response.status_code != 200:
                    print(f"v2 API 오류: {v2_response.text}")
                else:
                    v2_data = v2_response.json()
                    print(f"v2 API 결과: {v2_data}")
                
                # 모든 스페이스 조회해서 SPACE_KEY 찾기
                print("모든 스페이스에서 SPACE_KEY 찾기...")
                all_spaces_url = f"https://api.atlassian.com/ex/confluence/{cloud_id}/wiki/api/v2/spaces"
                all_spaces_response = requests.get(all_spaces_url, headers=headers, verify=False)
                if all_spaces_response.status_code == 200:
                    all_spaces_data = all_spaces_response.json()
                    print(f"전체 스페이스 수: {all_spaces_data.get('size', 0)}")
                    for space in all_spaces_data.get('results', []):
                        if 'SPACE_KEY' in space.get('name', '') or 'SPACE_KEY' in space.get('key', ''):
                            print(f"발견! 스페이스: {space.get('name')} (키: {space.get('key')})")
    except Exception as e:
        print(f"오류: {e}")
else:
    print("❌ 액세스 토큰이 없습니다. 이전 단계를 먼저 완료해주세요.")


## 6단계: 페이지 생성 테스트

지정한 폴더에 새로운 페이지를 생성해봅니다.


In [ ]:
import requests, json
from datetime import datetime

ACCESS_TOKEN = oauth_access_token

h = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Accept": "application/json",
    "Content-Type": "application/json",
}

create_url = f"https://api.atlassian.com/ex/confluence/{CLOUD_ID}/wiki/api/v2/pages?body-format=storage"
print("POST URL:", create_url)

payload = {
  "spaceId": SPACE_ID,
  "parentId": PARENT_ID,   # ✅ 부모 페이지 지정
  "title": f"API Test Page {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
  "body": {
    "representation": "storage",
    "value": "<p>첫 글! (created via API, inside folder)</p>"
  }
}

cr = requests.post(create_url, headers=h, json=payload, verify=False)
print("CREATE:", cr.status_code, cr.text[:400])

new_page = cr.json() if cr.status_code in (200, 201) else {}
new_id = new_page.get("id")
print("new page id:", new_id)

if new_id:
    print(f"✅ 새 페이지 URL: {ATLASSIAN_SITE}/wiki/spaces/SPACE_KEY/pages/{new_id}")
    # 새로 생성된 페이지 ID를 전역 변수로 저장
    TEST_PAGE_ID = new_id
    print(f"✅ 테스트 페이지 ID 저장됨: {TEST_PAGE_ID}")


## 7단계: 페이지 내용 읽기 및 업데이트 테스트

새로 생성된 페이지의 내용을 읽어오고 업데이트해봅니다.


In [ ]:
def get_headers():
    """API 요청용 헤더 생성"""
    return {
        "Authorization": f"Bearer {oauth_access_token}",
        "Content-Type": "application/json"
    }

def read_page_content(page_id: str):
    """페이지 내용 읽기 (v2 API 사용)"""
    print(f"=== 페이지 내용 읽기 (ID: {page_id}) ===")
    
    # v2 API로 페이지 내용 조회
    read_url = f"https://api.atlassian.com/ex/confluence/{CLOUD_ID}/wiki/api/v2/pages/{page_id}"
    params = {
        "body-format": "storage"  # HTML 형식으로 내용 가져오기
    }
    
    try:
        print("🔄 페이지 내용 조회 중... (v2 API)")
        response = requests.get(read_url, headers=get_headers(), params=params, verify=False)
        
        if response.status_code == 200:
            page_data = response.json()
            print(f"✅ 페이지 내용 조회 성공! (v2 API)")
            
            # 페이지 기본 정보
            print(f"\n📋 페이지 정보:")
            print(f"제목: {page_data.get('title')}")
            print(f"ID: {page_data.get('id')}")
            print(f"상태: {page_data.get('status')}")
            print(f"생성일: {page_data.get('createdAt')}")
            print(f"수정일: {page_data.get('version', {}).get('createdAt')}")
            print(f"버전: {page_data.get('version', {}).get('number')}")
            
            # 웹 URL
            web_url = f"{ATLASSIAN_SITE}{page_data.get('_links', {}).get('webui')}"
            print(f"웹 URL: {web_url}")
            
            # 페이지 내용
            body_content = page_data.get('body', {}).get('storage', {}).get('value', '')
            if body_content:
                print(f"\n📄 페이지 내용:")
                print("-" * 50)
                print(body_content)
                print("-" * 50)
                print(f"내용 길이: {len(body_content)} 문자")
            else:
                print("\n📄 페이지 내용: (비어있음)")
            
            return page_data
        else:
            print(f"❌ 페이지 내용 조회 실패: {response.status_code}")
            print(f"응답: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 페이지 내용 조회 중 오류: {e}")
        return None

def update_page_content(page_id: str, new_content: str = ""):
    """페이지 내용 업데이트 (v2 API 사용)"""
    print(f"=== 페이지 내용 업데이트 (ID: {page_id}) ===")
    
    # 1. 현재 페이지 정보 가져오기
    page_data = read_page_content(page_id)
    if not page_data:
        print("❌ 페이지 정보를 가져올 수 없어 업데이트를 중단합니다.")
        return False
    
    # 2. 현재 버전 정보
    current_version = page_data.get('version', {}).get('number', 1)
    print(f"현재 버전: {current_version}")
    
    # 3. 업데이트할 내용 준비
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    if not new_content:
        new_content = f"""
<h2>🧪 API 쓰기 권한 테스트</h2>
<p><strong>테스트 시간:</strong> {current_time}</p>
<p><strong>테스트 결과:</strong> ✅ 페이지 업데이트 성공!</p>
<p><strong>테스트 내용:</strong> OAuth 토큰을 사용한 Confluence API 쓰기 권한 확인</p>

<h3>📋 테스트 정보</h3>
<ul>
    <li>페이지 ID: {page_id}</li>
    <li>스페이스: SPACE_KEY</li>
    <li>API 버전: v2 (REST API)</li>
    <li>인증 방식: OAuth 2.0</li>
</ul>

<h3>🔧 기술 스택</h3>
<ul>
    <li>Python requests</li>
    <li>Atlassian Confluence REST API v2</li>
    <li>OAuth 2.0 Bearer Token</li>
</ul>

<p><em>이 페이지는 API 테스트를 위해 자동으로 생성/업데이트되었습니다.</em></p>
"""
    
    # 4. 페이지 업데이트 요청 (v2 API)
    update_url = f"https://api.atlassian.com/ex/confluence/{CLOUD_ID}/wiki/api/v2/pages/{page_id}"
    
    update_data = {
        "id": page_id,  # ✅ 필수: 페이지 ID
        "status": "current",  # ✅ 필수: 페이지 상태
        "version": {
            "number": current_version + 1
        },
        "title": page_data.get('title'),  # 제목 유지
        "body": {
            "representation": "storage",
            "value": new_content
        }
    }
    
    try:
        print("🔄 페이지 업데이트 중... (v2 API)")
        response = requests.put(update_url, headers=get_headers(), json=update_data, verify=False)
        
        if response.status_code == 200:
            updated_page = response.json()
            print(f"✅ 페이지 업데이트 성공! (v2 API)")
            print(f"새 버전: {updated_page.get('version', {}).get('number')}")
            
            # 전체 웹 URL 생성
            full_web_url = f"{ATLASSIAN_SITE}{updated_page.get('_links', {}).get('webui')}"
            print(f"전체 웹 URL: {full_web_url}")
            
            return True
        else:
            print(f"❌ 페이지 업데이트 실패: {response.status_code}")
            print(f"응답: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ 페이지 업데이트 중 오류: {e}")
        return False

# 테스트할 페이지 ID는 새로 생성된 페이지에서 가져옴

print("=== 페이지 내용 읽기 테스트 ===")
if 'oauth_access_token' in globals() and oauth_access_token and 'TEST_PAGE_ID' in globals() and TEST_PAGE_ID:
    # 1. 페이지 내용 읽기
    page_data = read_page_content(TEST_PAGE_ID)
    
    if page_data:
        print("\n🎉 페이지 내용 읽기가 성공했습니다!")
        
        # 2. 페이지 내용 업데이트 (선택사항)
        print("\n=== 페이지 내용 업데이트 테스트 ===")
        update_success = update_page_content(TEST_PAGE_ID, "")
        
        if update_success:
            print("\n🎉 페이지 업데이트도 성공했습니다!")
        else:
            print("\n❌ 페이지 업데이트가 실패했습니다.")
    else:
        print("\n❌ 페이지 내용 읽기가 실패했습니다.")
        print("   페이지 ID를 확인해주세요.")
else:
    if not oauth_access_token:
        print("❌ OAuth 액세스 토큰이 없습니다.")
        print("   이전 단계에서 OAuth 인증을 완료해주세요.")
    elif 'TEST_PAGE_ID' not in globals() or not TEST_PAGE_ID:
        print("❌ 새 페이지가 생성되지 않았습니다.")
        print("   이전 단계에서 새 페이지 생성을 먼저 완료해주세요.")


## 8단계: 테스트 결과 요약

페이지 읽기/쓰기 권한 테스트 결과를 종합합니다.


In [ ]:
# 테스트 결과 요약
print("\n" + "=" * 50)
print("📊 페이지 읽기/쓰기 권한 테스트 결과 요약")
print("=" * 50)

# 결과 확인
page_read_success = 'page_data' in locals() and page_data is not None
page_update_success = 'update_success' in locals() and update_success

print(f"페이지 내용 읽기: {'✅ 성공' if page_read_success else '❌ 실패'}")
print(f"페이지 내용 업데이트: {'✅ 성공' if page_update_success else '❌ 실패'}")

# 최종 결과
if page_read_success and page_update_success:
    print("\n🎉 모든 테스트가 성공했습니다!")
    print("   OAuth 토큰으로 Confluence API 읽기/쓰기 권한이 정상적으로 작동합니다.")
    print("\n📝 다음 단계:")
    print("   1. Confluence에서 업데이트된 페이지 확인")
    print("   2. 원하는 폴더에 페이지 생성")
    print("   3. 실제 프로젝트 문서 작성")
    
    # 테스트 페이지 URL 출력
    if page_read_success and 'TEST_PAGE_ID' in globals() and TEST_PAGE_ID:
        test_page_url = f"{ATLASSIAN_SITE}/wiki/spaces/SPACE_KEY/pages/{TEST_PAGE_ID}"
        print(f"\n🔗 테스트 페이지 URL: {test_page_url}")
        
elif page_read_success:
    print("\n⚠️  페이지 내용 읽기는 성공했지만, 업데이트가 실패했습니다.")
    print("   OAuth 스코프나 Confluence 권한 설정을 확인해주세요.")
    
else:
    print("\n❌ 페이지 내용 읽기부터 실패했습니다.")
    print("   OAuth 토큰이나 페이지 ID를 확인해주세요.")

print("\n" + "=" * 50)
print("테스트 완료!")
print("=" * 50)
